### T1 : 엑셀 가공

- 엑셀 데이터 열기

In [ ]:
import win32com.client
import pandas as pd
import pyautogui as pg
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By


excel = win32com.client.Dispatch('Excel.Application')
excel.Visible = True
wb = excel.Workbooks.Open('D:\작업방\RPA 과제\GLD_진로설계서_테스트2.xlsx')

In [ ]:
ws1 = wb.Worksheets('주소록')
ws2 = wb.Worksheets('설문')
ws3 = wb.Worksheets('진로설계서')

- 총 인원수 카운트

In [ ]:
i = 1
cell_value = ws1.Cells(i, 1).Value

while cell_value != None :
      i+=1
      cell_value = ws1.Cells(i,1).Value

cnt = i-2

print(cnt)

### T2. EP 접속

- EP 계정 정보 입력(ID, PASS)

In [ ]:
my_id = pg.prompt(title = 'ID_PASS', text = '아이디를 입력하세요', default = '')
my_pw = pg.password(title='EP_PASS', text='패스워드를 입력하세요', default='')

- EP 접속

In [ ]:
url = 'http://swpsso.posco.net/idms/U61/jsp/login/login.jsp'

driver = webdriver.Edge('D:\작업방\msedgedriver')
driver.get(url)
driver.maximize_window()


- 계정 입력

In [ ]:
driver.find_element(By.XPATH,'//*[@id="username"]').send_keys(my_id)
driver.find_element(By.XPATH,'//*[@id="password"]').send_keys(my_pw)
driver.find_element(By.XPATH,'//*[@id="loginsubmit"]').click()
driver.implicitly_wait(10) #드라이버가 로딩되는데 10초까지 대기, 빨리 로딩되면 다음 코드 진행

- 예외처리 : 기존 로그인 경고창이 열릴 경우 alert accept 처리

In [ ]:
pg.sleep(5)
try:  
    alert=driver.switch_to.alert
    alert.accept()
except:
    print("기존 로그인 경고창 없음. 계속 진행.")  # 경고창이 열리지 않을 경우

- 예외처리 : 현재 활성화된 드라이버 창 확인하여 SSO Notice 창 닫기 

In [ ]:
pg.sleep(2) #슬립이 없으면 에러

try:
      for i in reversed(range(len(driver.window_handles))):
            driver.switch_to.window(driver.window_handles[i])
            if driver.title != 'EP(Enterprise Portal)':
                  driver.close()
except:
    print("팝업창 없음-계속 진행")  # 팝업창이 하나도 없을 경우

### T3. EP 메일화면 접속

- 메일 화면 열기

In [119]:
pg.sleep(5) #로그인 되는 시간을 충분히 줌

driver.switch_to.window(driver.window_handles[-1]) #최근창으로 전환
driver.find_element(By.XPATH,'//*[@id="533982"]').click()


- 메일 쓰기 화면 열기

In [ ]:
# pg.sleep(3)

# driver.switch_to.window(driver.window_handles[-1]) #최근창으로 전환
# driver.find_element(By.XPATH,"//*[@id='Lnb']/div[1]/a").click()


### T4. 메일 보내기
- 반복작업(메일쓰기 -> 설계서 양식 만들기 -> 메일 주소,제목 넣기 -> 메일 본문에 양식 넣기 -> 메일 보내기)

In [121]:
for c in range(cnt): #3이라면 0~2

      pg.sleep(3)
      # 메일 쓰기화면 열기      
      driver.switch_to.window(driver.window_handles[-1]) #최근창으로 전환
      driver.find_element(By.XPATH,"//*[@id='Lnb']/div[1]/a").click()

      pg.sleep(3)
      sub = '님께서 「GLD 예비과정_커리어설계」 에서 작성한 진로설계서입니다.'

      name = ws1.Cells(c+2,1).Value #성명
      mail_add = ws1.Cells(c+2,3).Value #이메일       

      ws3.Cells(3,1).Value = name + sub #성명 + 인사말

      #설계서 내용 옮기기 : 양식 만들기
      for cell in range(0,4):
            ws3.Cells(cell+6,2).Value = ws2.Cells(c+2,cell+12).value

      driver.switch_to.window(driver.window_handles[-1]) #최근창으로 전환
      driver.find_element(By.XPATH,'//*[@id="token-input-send_to"]').send_keys(mail_add) # 받는 사람

      #메일 주소는 엔터를 해줘야 함
      pg.sleep(1)
      win = pg.getWindowsWithTitle("메일쓰기")[0]
      win.activate()

      pg.press('enter') #이메일 주소 완성
      pg.sleep(1)

      pg.sleep(3)

      #메일 정보, 본문 입력

      mail_sub = name + sub #성명 + 인사말

      pg.sleep(1)

      driver.find_element(By.XPATH,'//*[@id="write_send_info"]/table[2]/tbody/tr[1]/td/input').send_keys(mail_sub) # 제목

      ws3.Range('A1:B9').Copy()

      win = pg.getWindowsWithTitle('메일쓰기')[0]
      win.activate()

      pg.press('tab')
      pg.sleep(1)
      pg.hotkey('ctrl','v')

      pg.sleep(1)
      pg.press('tab',presses=11)
      pg.press('enter')

      #메일 보내기
      pg.sleep(2)
      driver.find_element(By.XPATH,'//*[@id="memo_content"]/div[1]/ul[1]/li[1]/a').click()

      #메일 발송 확인창 확인 버튼 클릭- alert창으로 처리
      pg.sleep(5)
      try:  # 메일 발송 확인창이 열릴 경우 alert accept 처리
            alert=driver.switch_to.alert
            alert.accept()
      except:
            print("메일 발송 확인창 없음. 계속 진행.")

      pg.sleep(3) #배치파일로 실행할 때는 모든 창이 닫기므로 메일을 보내는 시간이 필요함

      driver.switch_to.window(driver.window_handles[-1]) #최근창으로 전환
      # driver.close()